In [21]:
from spacy_preprocessor import SpacyPreprocessor
import sqlalchemy as db
import pandas as pd
import scattertext as st

# Import Database

In [2]:
engine = db.create_engine('postgresql://localhost:5432/lyricsdb')
connection = engine.connect()
metadata = db.MetaData()
artists = db.Table('artists', metadata, autoload=True, autoload_with=engine)
songs = db.Table('songs', metadata, autoload=True, autoload_with=engine)

In [3]:
query = 'SELECT * FROM songs;'
song_df_db = pd.read_sql(query, connection)

In [4]:
query = 'SELECT * FROM artists;'
artist_df_db = pd.read_sql(query, connection)

# Grab Songs

In [5]:
normal_songs = song_df_db[(song_df_db["lyrics"].str.len() <= 3500) & (song_df_db["lyrics"].str.len() >= 400)]

In [13]:
normal_songs.head()

,artist_id,song_title,lyrics
2,347,Okie from Muskogee,We don't smoke marijuana in Muskogee\nWe don't...
3,347,Mama Tried,The first thing I remember knowing\nWas a lone...
4,347,One Day at a Time,"I'm only human, I'm just a man\nHelp me to bel..."
5,347,Sing Me Back Home,The warden led a prisoner down the hallway to ...
6,347,That’s the Way Love Goes,I've been throwing horseshoes over my left sho...


In [74]:
artist_1 = "Bruce Springsteen"
artist_2 = "Merle Haggard"

In [75]:
artist_1_id = artist_df_db[artist_df_db["artist_name"]==artist_1]['index'].tolist()[0]
artist_2_id = artist_df_db[artist_df_db["artist_name"]==artist_2]['index'].tolist()[0]
artist_list = [artist_1_id, artist_2_id]

In [76]:
two_artist_dataframe = normal_songs[normal_songs["artist_id"].isin(artist_list)]

In [77]:
two_artist_dataframe["artist_id"] = two_artist_dataframe["artist_id"].astype(str)
two_artist_dataframe

,artist_id,song_title,lyrics
2,347,Okie from Muskogee,We don't smoke marijuana in Muskogee\nWe don't...
3,347,Mama Tried,The first thing I remember knowing\nWas a lone...
4,347,One Day at a Time,"I'm only human, I'm just a man\nHelp me to bel..."
5,347,Sing Me Back Home,The warden led a prisoner down the hallway to ...
6,347,That’s the Way Love Goes,I've been throwing horseshoes over my left sho...
...,...,...,...
24956,1310,Darkness on the Edge of Town [MTV Plugged],They're still racing out at the Trestles\nBut ...
24957,1310,Nebraska (1982),I saw her standing on her front lawn just twir...
24958,1310,"4th of July, Asbury Park (Sandy) [Magic Tour H...","Sandy, the fireworks are hailing over Little E..."
24959,1310,The Ghost of Tom Joad [Magic Tour Highlights],Men walking along the railroad tracks\nGoing s...


# Remove duplicate songs

In [78]:
song_titles_dict = {}
index_removal_list = []
for i, row in two_artist_dataframe.iterrows():
    artist_id = row["artist_id"]
    if artist_id in song_titles_dict.keys():  
        for song in song_titles_dict[artist_id]:
            if row["song_title"] in song or song in row["song_title"]:
                # print(row["song_title"])
                index_removal_list.append(i)
        else:
            song_titles_dict[artist_id].append(row["song_title"])
    else: 
        song_titles_dict[artist_id] = [row["song_title"]]

In [79]:
two_artist_dataframe.drop(index_removal_list, inplace=True, errors='ignore',)
two_artist_dataframe

,artist_id,song_title,lyrics
2,347,Okie from Muskogee,We don't smoke marijuana in Muskogee\nWe don't...
3,347,Mama Tried,The first thing I remember knowing\nWas a lone...
4,347,One Day at a Time,"I'm only human, I'm just a man\nHelp me to bel..."
5,347,Sing Me Back Home,The warden led a prisoner down the hallway to ...
6,347,That’s the Way Love Goes,I've been throwing horseshoes over my left sho...
...,...,...,...
24938,1310,Say Sons,"Well, I got a little girl down by the river\nN..."
24941,1310,Without You,"Monday, I go to work\nTuesday, can't find my s..."
24943,1310,Can’t Stand Losing You,I've called you so many times today\nAnd i gue...
24944,1310,Sociedade Alternativa,Viva! Viva!\nViva a Sociedade Alternativa!\nVi...


# Remove Duplicate Lines in Song

# Create the scattertext

In [86]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

/Users/ryanwerth/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_extraction.stop_words module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_extraction.text. Anything that cannot be imported from sklearn.feature_extraction.text is now part of the private API.
  warnings.warn(message, FutureWarning)


In [87]:
corpus = st.CorpusFromPandas(two_artist_dataframe,
                             category_col='artist_id',
                             text_col='lyrics',
                             nlp=st.whitespace_nlp_with_sentences
                            ).build().remove_terms(ENGLISH_STOP_WORDS, ignore_absences=True)

In [88]:
html = st.produce_scattertext_explorer(
        corpus,
        category='347',
        category_name='Merle Haggard',
        not_category_name='Bruce Springsteen',
        minimum_term_frequency=10,
        pmi_threshold_coefficient=5,
        width_in_pixels=1000,
        metadata=two_artist_dataframe['song_title'],
        )

In [89]:
open('scattertext_demo.html', 'wb').write(html.encode('utf-8'));

# Put in Function

In [ ]:
def remove_duplicate_songs(artist_dataframe):
    song_titles_dict = {}
    index_removal_list = []
    for i, row in artist_dataframe.iterrows():
        artist_id = row["artist_id"]
        if artist_id in song_titles_dict.keys():  
            for song in song_titles_dict[artist_id]:
                if row["song_title"] in song or song in row["song_title"]:
                    # print(row["song_title"])
                    index_removal_list.append(i)
            else:
                song_titles_dict[artist_id].append(row["song_title"])
        else: 
            song_titles_dict[artist_id] = [row["song_title"]]

In [ ]:
def create_scatter_text(artist_1, artist_2, artist_dataframe, song_dataframe, file_name):
    